In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, pipeline
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer
from datasets import load_dataset

/home/trevor/TR/topic-responder-venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.2+cu121 with CUDA 1201 (you have 1.12.1+cu102)
    Python  3.9.18 (you have 3.9.16)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [2]:
#b3_tokenizer = AutoTokenizer.from_pretrained("./rg_output_vicuna/checkpoint-8255")
#b3_model = AutoModelForCausalLM.from_pretrained("./rg_output_vicuna/checkpoint-8255")
b3_tokenizer = AutoTokenizer.from_pretrained("./rg_output/checkpoint-12354")
b3_model = AutoModelForCausalLM.from_pretrained("./rg_output/checkpoint-12354",torch_dtype=torch.float32)
b3_model.to('cuda:0')

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.41s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
      

In [5]:
def form_func(sample):
    if 'agent_1' in sample['response']:
        p_in = 'agent_1'
        not_p_in = 'agent_2'
    else:
        p_in = 'agent_2'
        not_p_in = 'agent_1'

    messages = [
        {
        "role":"system",
        "content": f"You are participating in the conversation. You are specifically {p_in}."
        },
        {
        "role": "user",
        "content": f"Generate the next conversation turn for {p_in} responding to {not_p_in} in this conversation: {sample['dialogue']} Limit the generated response to 1-2 sentences."
        },
        # {
        # "role": "assistant",
        # "content": f"{p_in}:{sample['response']}"
        # }
    ]

    prompt = b3_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [9]:
# p_in = 'person 1'
# not_p_in = 'person 2'

# user_in = 'person 2: I love watching movies\n'

# inputs = b3_tokenizer(user_in + f'{p_in}:', return_tensors="pt").to('cuda:0')
                
# generate_ids = b3_model.generate(inputs.input_ids, max_length=500, temperature=0.8, top_k=50, top_p = 0.85)
# b3_out = b3_tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
# b3_out = b3_out.split(f"{p_in}:")[-1]

p_in = 'agent_1'
not_p_in = 'agent_2'

user_in = 'agent_2: Hello.'

# llama_in = f'<s>[INST] <<SYS>>\nYou are a person participating in a conversation. You are specifically {p_in}. <</SYS>>\nGenerate the next conversation turn for {p_in} responding to {not_p_in} in this conversation: {user_in} Limit the generated response to 1-2 sentences [/INST] {p_in}:'
llama_in = form_func({'dialogue':"agent_2:Yeah linkin park use to be one of my favorites. Do you like them?  agent_1:I do, I saw them live once! Did you know their album hybrid theory was the best selling album of the 21st century? agent_2:Wow that's awesome, so sorry how things turned out for them. Even after 40 years the dark side of the moon is still one of the best selling albums of 2014. agent_1:They were amazing. Very sad about what happened.  agent_2:Did you here about the 100 year old twins in Japan?  agent_1:Yea they released a rap album at that age! Impressive. agent_2:Very i wouldn't mind watching them perform it but since it was in 1992 i doubt they are still around. agent_1:Probably not. Are you into comedy? agent_2:Only when it's funny, haha. what about you, ever seen 3rd rock from the sun?  agent_1:I have not! They won the golden globe in 1997 for best comedy series! agent_2:Yeah they beat friends and fraiser. agent_1:They also beat seinfeld! That is impressive and I never would have guessed. agent_2:Right! You know there is a genre called bromantic comedies? agent_1:Those are my favorite. Especially Harold and Kumar and the Hangover. agent_2:The hangovers were crazy!! Do you like cars? agent_1:I do a little bit. I love how drive throughs were originally for soldiers. agent_2:Yeah that amazes me i would have thought they would have been around in the 50's. I do appreciate why they did it though. You know Ryan Stiles dropped out of high school to pursue comedy.  agent_1:Demetri Martin also dropped out of school to pursue comedy! agent_2:Do you know if he is any good i don't recognize his name? Do you like Bill Murray i think he's strange but funny?", 'response':'agent_1: HELLO THERE HOW ARE YOU?'})
# print(llama_in)
generate_ids = b3_tokenizer(llama_in, max_length=1224, return_tensors='pt', truncation=True).to('cuda:0')
b3_out = b3_model.generate(generate_ids.input_ids, max_new_tokens=100, temperature=0.8, top_k=50, top_p = 0.85)
b3_out = b3_tokenizer.batch_decode(b3_out, skip_special_tokens=True)[0].split('[/INST]')[-1]
b3_out = b3_out.replace('agent_1:', '')
b3_out = b3_out.replace('agent_2:', '')

print(b3_out)

 I think he is funny. I loved Groundhog day. 
